In [ ]:
# Copyright 2022 Diabwellness.ai, Inc.
# All rights reserved

# Risk Profiling

Notebook to cluster patients using VaDER network and profile them with potential risk factors. This type of clustering is based on the longitudinal/time-series data of the patients unlike k-means which is a crossectional clustering.

We use the following tables for this analysis: 
- `measurement_details`: contains the various measurements corresponding to every appointments
- `patient_details`: contains the age, gender and diabetic duration of the patients

In [ ]:
import os
import time
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bokeh import core, io, palettes, models, layouts
from bokeh.plotting import output_file, figure
from bokeh.models import BoxAnnotation, LinearAxis, Range1d
from bokeh.io import show, output_notebook

io.reset_output()
io.output_notebook()

import tensorflow as tf

tf.config.run_functions_eagerly(False)
# from diabwellness.VaDER.VaDER.vader import VADER
from VaDER.vader import VADER

# import VaDER

from diabwellness.utils.data_utils import (
    preprocess_measurements,
    measurement_aggregator,
    combine_patient_details,
    map_complications,
    map_patient_types,
)
from diabwellness.utils.plot_utils import (
    create_metrics_plot,
    display_factorial_planes,
    display_parallel_coordinates_centroids,
)

# Change jupyter notebook to full width for extra visualization space
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option("display.max_rows", 50000)
pd.set_option("display.max_columns", None)

In [ ]:
# read in the patient details:
pat_df = pd.read_csv(
    "../database/patient_details.tsv",
    sep="\t",
    index_col=False,
    parse_dates=["created_time"],
)
pat_df.info()

colnames = [
    "ID",
    "APPOINT_ID",
    "BMI",
    "BP",
    "COMPLAINTS",
    "CREATED_BY",
    "CREATED_DATE",
    "DIAGNOSIS",
    "HEIGHT",
    "LOCATION_ID",
    "NFID",
    "STATUS",
    "TEMPERATURE",
    "UPDATED_BY",
    "UPDATED_DATE",
    "WC",
    "WEIGHT",
    "PATIENT_TYPE",
    "A1C",
    "DIA_BP",
    "DURATION_TT",
    "FS",
    "NOTES",
    "PP",
    "PULSE",
    "REVIEW_DATYS",
    "ADMISSION_REQUIRED",
    "REVIEW_DATE",
    "LAB_FOR_NEXT_VISIT",
]
meas_df = pd.read_csv(
    "../database/measurement_details.tsv",
    sep="\t",
    names=colnames,
    header=None,
    parse_dates=["CREATED_DATE", "UPDATED_DATE"],
)
meas_df.info()

In [ ]:
# preprocess the measurement details:
meas_df = preprocess_measurements(meas_df)
# extract the diabetic complications and patient types:
meas_df = map_complications(meas_df)
meas_df = map_patient_types(meas_df)
meas_df.head(10)

In [ ]:
meas_filt_df = (
    meas_df.groupby(["NFID"]).apply(measurement_aggregator).dropna(subset=["A1C"])
)
meas_filt_df = combine_patient_details(meas_filt_df, pat_df)

meas_filt_df.head()

In [ ]:
meas_filt_df["COMPLICATIONS"].value_counts()

In [ ]:
meas_filt_df.loc[meas_filt_df["PATIENT_TYPE"] == {"NON-DM", "DM"}].head(100)

In [ ]:
meas_filt_df["PATIENT_TYPE"].value_counts()

In [ ]:
meas_filt_df.shape

## Visualize the Patient measurements:

In [ ]:
# visualize A1c, FS, PP and other characteristics of a patient over time:
select_cols = [
    "A1C",
    "FS",
    "PP",
    "CREATED_DATE",
    "DURATION",
    "PATIENT_TYPE",
    "AGE",
    "GENDER",
]
explode_cols = ["A1C", "FS", "PP", "CREATED_DATE"]
a1c_cond = meas_filt_df["A1C_counts"] >= 2

plot_list = []
# print(input_df.iloc[2].to_frame().T.explode(cols).set_index('CREATED_DATE'))

for i in range(10, 15):
    input_df = (
        meas_filt_df.loc[a1c_cond][select_cols]
        .reset_index()
        .iloc[i]
        .to_frame()
        .T.explode(explode_cols)
        .set_index("CREATED_DATE")
    )
    plot = create_metrics_plot(
        input_df=input_df,
        varea_plots={
            "PP": ["wheat", "sugar", 0.0],
            "FS": ["chocolate", "sugar", 0.0],
        },
        line_plots={"A1C": "black"},
        extra_y_ranges={"sugar": Range1d(start=0, end=500)},
        title=f"Measurements for NFID {input_df['NFID'][0]}\
            diabetic duration (in days): {input_df['DURATION'][0].astype('int')}\
            patient type: {input_df['PATIENT_TYPE'][0]}\
            age: {input_df['AGE'][0].astype('int')}\
            gender: {input_df['GENDER'][0]}",
        stacked_area_plots=False,
    )
    plot.legend.orientation = "horizontal"
    plot.y_range = Range1d(start=0, end=15)
    plot_list.append(plot)


# Link the x-axis and y-axis
xr = plot_list[0].x_range
yr = plot_list[0].y_range
for pl in plot_list:
    pl.x_range = xr
    pl.y_range = yr
combined_plot = layouts.column(plot_list)
show(combined_plot)

## Train with VaDER:

In [ ]:
a1c_cond = meas_filt_df["A1C_counts"] == 3
meas_filt_df.loc[a1c_cond].shape
meas_filt_df["A1C_counts"].value_counts()

In [ ]:
a1c_cond = meas_filt_df["A1C_counts"] >= 2
X_train = np.array(meas_filt_df.loc[a1c_cond]["A1C_first_two_values"].tolist())
# X_train shape should be (n_patients, n_timesteps, n_variables)
# Note: All X_train[i,j] for which W_train[i,j] == 0 are treated as missing (i.e. their specific value is ignored)
X_train = np.expand_dims(X_train, axis=2)
X_train.shape

In [ ]:
X_train

In [ ]:
# normalize (better for fitting)
# for i in np.arange(X_train.shape[2]):
#     print(i)
#     X_train[:,:,i] = (X_train[:,:,i] - np.mean(X_train[:,:,i])) / np.std(X_train[:,:,i])
# X_train

In [ ]:
# Note: y_train is used purely for monitoring performance when a ground truth clustering is available.
# It can be omitted if no ground truth is available.
from pathlib import Path
Path("./vader_checkpoints").mkdir(parents=True, exist_ok=True)
save_path = os.path.join("./vader_checkpoints", "vader_a1c.ckpt")

vader = VADER(
    X_train=X_train,
    W_train=None,
    y_train=None,
    save_path=save_path,
    n_hidden=[12, 2],
    k=4,
    learning_rate=1e-3,
    output_activation=None,
    recurrent=True,
    cell_type="LSTM",
    batch_size=64,
)
# pre-train without latent loss
start = time.time()
vader.pre_fit(n_epoch=50, verbose=True)
# train with latent loss
vader.fit(n_epoch=50, verbose=True)
end = time.time()
print("Elapsed: ", end - start)

In [ ]:
# get the clusters
c = vader.cluster(X_train)
# get the re-constructions
p = vader.predict(X_train)
# compute the loss given the network
l = vader.get_loss(X_train)

print("clusters: ", c)
# print("reconstructions: ", p)
# print("losses: ", l)

In [ ]:
# add the cluster index to the original dataframe:
meas_filt_df["CLUSTER_INDEX"] = np.nan
a1c_cond = meas_filt_df["A1C_counts"] >= 2

cluster_series = pd.Series(c, index=meas_filt_df.loc[a1c_cond].index, dtype="int")
meas_filt_df.loc[a1c_cond, "CLUSTER_INDEX"] = meas_filt_df.loc[
    a1c_cond, "CLUSTER_INDEX"
].fillna(cluster_series)

meas_filt_df.head()

In [ ]:
select_cols = [
    "HEIGHT",
    "WEIGHT",
    "BMI",
    "BP",
    "DIA_BP",
    "PULSE",
    "A1C_first_two_values",
    "FS",
    "PP",
    "DURATION",
    "PATIENT_TYPE",
    "AGE",
    "GENDER",
    "CLUSTER_INDEX",
]


def process_cluster_df(cluster_df):
    cluster_df["A1C_first"] = cluster_df["A1C_first_two_values"].apply(lambda x: x[0])
    cluster_df["A1C_second"] = cluster_df["A1C_first_two_values"].apply(lambda x: x[1])
    cluster_df["FS"] = cluster_df["FS"].apply(lambda x: pd.Series(x).dropna().mean())
    cluster_df["PP"] = cluster_df["PP"].apply(lambda x: pd.Series(x).dropna().mean())
    #     cluster_df['PATIENT_TYPE'] = cluster_df['PATIENT_TYPE'].map('tuple').astype('category').cat.codes
    #     cluster_df['GENDER'] = cluster_df['GENDER'].astype('category').cat.codes

    cat_cols = ["PATIENT_TYPE", "GENDER", "A1C_first_two_values"]
    df = cluster_df.drop(columns=cat_cols).dropna().copy()
    df_norm = (df - df.mean()) / df.std()
    X_scaled = df_norm.values
    return X_scaled


X_scaled = process_cluster_df(meas_filt_df.loc[a1c_cond, select_cols])
X_scaled.shape

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(X_scaled)
# pca.components_  # we only take the first two features.

# Transfor the scaled data to the new PCA space
X_reduced = pca.transform(X_scaled)

# centres_reduced = pca.transform(kmeans.cluster_centers_)

# fig = plt.figure()

display_factorial_planes(
    X_reduced, 2, pca, [(0, 1)], illustrative_var=X_scaled[:, 10], alpha=0.8
)

In [ ]:
meas_filt_df["CLUSTER_INDEX"].value_counts()

In [ ]:
meas_filt_df.head(10)

In [ ]:
PATIENT_TYPES = {
    ("DM",): "DM",
    ("NON-DM",): "NON-DM",
    ("NON-MS",): "NON-MS",
    (
        "THY",
        "Hypothyroid",
        "Hypothyroidism",
        "Thyroid",
    ): "THY",
    ("IDDM",): "IDDM",
    ("GDM",): "GDM",
    (
        "InflDisorder",
        "Inf",
    ): "InflDisorder",
    (
        "HT",
        "HTN",
    ): "HT",
    #     ("Allergy",): "Allergy",
    #     ("Anemia",): "Anemia",
    #     ("Cancer",): "Cancer",
    (
        "Hlip",
        "Hyperlipidemia",
    ): "Hlip",
}

COMPLICATION_TYPES = {
    (
        "CAD",
        "Angina",
        "Unstable angina",
        "ASMI",
        "AWMI",
        "IWMI",
        "PTCA",
        "CABG",
        "heart attack",
        "infarct",
        "ischemia",
    ): "CAD",
    (
        "Stroke",
        "hemiplegia",
        "hemiparesis",
        "cerebral infarct",
        "VBI",
        "vertebrobasilar infarct",
        "pontine infarct",
        "PICA",
        "AICA",
        "cerebellar infarct",
    ): "CVD",
    (
        "Intermittent claudication",
        "pain legs on walking",
        "pain calf muscles on walking",
    ): "PVD",
    (
        "PN",
        "painful peripheral Neuropathy",
        "numbness",
        "tingling",
        "burning feet",
        "numb feet",
        "numb limbs",
        "DPN",
    ): "DPN",
    (
        "DKD",
        "CKD",
        "Microalbuminuria",
        "Macroalbuminuria",
        "DN",
        "Nephropathy",
        "diabetic nephropathy",
    ): "DN",
    (
        "DR",
        "PDR",
        "NPDR",
        "Diabetic retinopathy",
    ): "DR",
    (
        "Foot ulcer",
        "bleb",
        "wound",
        "amputee",
        "fissure",
        "callus",
        "corn",
        "DFU",
        "Nonhealing ulcer",
    ): "DFU",
}

from collections import Counter
import itertools


def cluster_aggregator(group):
    output_dict = {}

    def extract_set_fractions(series, types_dict):
        counts_dict = Counter(
            {given_type: 0 for given_type in set(types_dict.values())}
        )
        cluster_list = series.dropna().apply(lambda x: list(x)).tolist()
        merged_list = list(itertools.chain(*cluster_list))
        counts_dict.update(merged_list)
        return {
            f"{series.name}_{k}": f"{v}/{series.size}" for k, v in counts_dict.items()
        }

    # columns where average value should be taken
    mean_cols = ["HEIGHT", "WEIGHT", "BMI", "BP", "DIA_BP", "PULSE", "DURATION", "AGE"]
    output_dict = {col: group[col].mean() for col in mean_cols}

    # count of new T2DM patients in each cluster
    output_dict["NEW_T2DM"] = group["NEW_T2DM"].sum()

    # first, second and last values of A1C values are considered
    output_dict["A1C_first"] = (
        group["A1C_first_two_values"].apply(lambda x: x[0]).mean()
    )
    output_dict["A1C_second"] = (
        group["A1C_first_two_values"].apply(lambda x: x[1]).mean()
    )
    output_dict["A1C_last"] = (
        group["A1C"].apply(lambda x: pd.Series(x).dropna().iloc[-1]).mean()
    )

    # get the size of patients in each cluster:
    output_dict["CLUSER_SIZE"] = group.shape[0]

    # first and last values of FF and PP  are considered
    #     print(group['FS'])
    #     output_dict['FS_first'] = group['FS'].apply(lambda x: pd.Series(x).dropna().iloc[0]).mean()
    #     output_dict['FS_last'] = group['FS'].apply(lambda x: pd.Series(x).dropna().iloc[-1]).mean()
    #     output_dict['PP_first'] = group['PP'].apply(lambda x: pd.Series(x).dropna().iloc[0]).mean()
    #     output_dict['PP_last'] = group['PP'].apply(lambda x: pd.Series(x).dropna().iloc[-1]).mean()

    # combine COMPLICATIONS, PATIENT_TYPE into a single set and then to proportions:
    output_dict.update(
        extract_set_fractions(group["COMPLICATIONS"], COMPLICATION_TYPES)
    )
    output_dict.update(extract_set_fractions(group["PATIENT_TYPE"], PATIENT_TYPES))

    return pd.Series(output_dict)


# check for Waist circumference (WC) from Patient details

In [ ]:
cluster_df = meas_filt_df.groupby(["CLUSTER_INDEX"]).apply(cluster_aggregator)
cluster_df

In [ ]:
from pandas.plotting import parallel_coordinates
import seaborn as sns

palette = sns.color_palette("bright", 10)

# ax = display_parallel_coordinates_centroids(cluster_df, 10)

cluster_df = (cluster_df - cluster_df.mean()) / cluster_df.std()
cluster_df["cluster"] = cluster_df.index

# Create the plot
fig = plt.figure(figsize=(30, 10))
fig.suptitle("Parallel Coordinates plot for the Centroids")
fig.subplots_adjust(top=0.9, wspace=0)

# Draw the chart
ax = parallel_coordinates(cluster_df, "cluster", color=palette)
plt.xticks(rotation=50)

In [ ]:
# NOTES:
# variables for vader: A1c, age, duration, BMI, FS, PP, BP, DIA_BP, PULSE
# report ratios based on cluster size as well as total patients in each compl/pat_type

# data based on time periods: (2017' March to 2021)? choose n_timesteps average? year end timesteps?
# arbitrary value of timesteps?

# P1: t1,t2, t3 -> t1, t2, t3, t4, t5
#     A, 0, A
#     F, F, F
#     0, 0 ,0

# P2: t1, t2, t3, .. t10 -> t1, t2, t3, t4, t5
#     0, A, 0,    .. A
#     0, F, 0,   ..  0
#     P, P, P,  . .  P

# P3: t1
#     0
#     F
#     P


# X_train = n_patients x n_timesteps x n_variables
#         = N x 5 x 3